In [1]:
import pandas as pd

In [2]:
file_path = '/content/raw_dataset.csv'

In [3]:
df = pd.read_csv(file_path)

In [4]:
df.sample(10)

,ID,Name,Age,Gender,Salary,Department,Joining_Date,Performance_Score,Remarks
38,39,David,25.0,NaN,50000.0,HR,2021-03-20,1.0,Excellent
32,33,Alice,30.0,M,80000.0,it,2019-07-01,3.0,NaN
209,169,Alice,35.0,Unknown,80000.0,it,2022-11-11,3.0,Average
41,42,NaN,45.0,M,40000.0,finance,2022-11-11,100.0,Needs Improvement
167,168,Charlie,30.0,Female,80000.0,HR,2020-01-15,2.0,Average
134,135,Eva,20.0,NaN,80000.0,NaN,2019-07-01,5.0,Average
156,157,Eva,NaN,F,40000.0,Finance,NaN,2.0,NaN
146,147,Alice,50.0,F,NaN,IT,2021-03-20,100.0,Excellent
31,32,Alice,50.0,NaN,60000.0,IT,2022-11-11,100.0,Good
45,46,David,50.0,Unknown,40000.0,Finance,2021-03-20,NaN,NaN


# 1. Standarize the data

In [5]:
df["Gender"] = df["Gender"].str.strip().str.capitalize().replace({"Male":"M", "Female": "F"})

In [6]:
df["Name"] = df["Name"].str.strip().str.capitalize()

In [7]:
df["Department"] = df["Department"].str.strip().str.capitalize().replace({"It":"IT", "Hr":"HR"})

# 2. Handling Missing Data

NOTE: If in any column, the missing data covers over 50% than ideally the column is discarded.

In [8]:
df.isnull().sum()

,0
ID,0
Name,25
Age,13
Gender,36
Salary,39
Department,34
Joining_Date,47
Performance_Score,30
Remarks,33


In [9]:
thres = len(df) * 0.5

In [10]:
df = df.dropna(axis = 1, thresh = thres)

In [11]:
for col in df.columns:
  if df[col].dtype in ["float64", "int64"]:
    df[col] = df[col].fillna(df[col].median())
  else:
    df[col] = df[col].fillna(df[col].mode()[0])

In [12]:
# Convert Joining_Date to date and time
df["Joining_Date"] = pd.to_datetime(df["Joining_Date"])

In [13]:
df.sample(10)

,ID,Name,Age,Gender,Salary,Department,Joining_Date,Performance_Score,Remarks
38,39,David,25.0,F,50000.0,HR,2021-03-20,1.0,Excellent
7,8,Alice,35.0,Unknown,40000.0,Marketing,2020-01-15,100.0,Excellent
165,166,Alice,50.0,Unknown,60000.0,Finance,2019-07-01,2.0,Average
107,108,Bob,40.0,F,60000.0,HR,2022-11-11,100.0,Average
195,196,Alice,40.0,F,60000.0,HR,2022-11-11,1.0,Good
62,63,Bob,50.0,M,30000.0,HR,2020-01-15,4.0,Average
150,151,David,20.0,M,50000.0,IT,2019-07-01,5.0,Excellent
170,171,Alice,40.0,F,50000.0,IT,2020-01-15,3.0,Good
78,79,David,35.0,Unknown,30000.0,Finance,2019-07-01,2.0,Needs Improvement
183,184,Eva,30.0,F,40000.0,IT,2021-03-20,1.0,Average


In [14]:
df.isnull().sum()

,0
ID,0
Name,0
Age,0
Gender,0
Salary,0
Department,0
Joining_Date,0
Performance_Score,0
Remarks,0


In [15]:
df.dtypes

,0
ID,int64
Name,object
Age,float64
Gender,object
Salary,float64
Department,object
Joining_Date,datetime64[ns]
Performance_Score,float64
Remarks,object


# 3. Ensuring Correct Datatypes

In [16]:
df["Age"] = df["Age"].astype("int64")
df["Salary"] = df["Salary"].astype("int64")
df["Performance_Score"] = df["Performance_Score"].astype("int64")

In [17]:
df.dtypes

,0
ID,int64
Name,object
Age,int64
Gender,object
Salary,int64
Department,object
Joining_Date,datetime64[ns]
Performance_Score,int64
Remarks,object


# 4. Outliers Removal using IQR

In [18]:
def remove_outliers_IQR(data, column):
  Q1 = data[column].quantile(0.25)
  Q3 = data[column].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

In [19]:
df = ( df
      .pipe(remove_outliers_IQR, column = "Age")
      .pipe(remove_outliers_IQR, column = "Salary")
      .pipe(remove_outliers_IQR, column = "Performance_Score")
      )

In [20]:
df = df.drop_duplicates()

In [22]:
df.sample(10)

,ID,Name,Age,Gender,Salary,Department,Joining_Date,Performance_Score,Remarks
12,13,Charlie,50,Unknown,60000,Finance,2022-11-11,1,Excellent
133,134,Charlie,35,F,50000,HR,2020-01-15,5,Average
50,51,Eva,40,M,80000,Marketing,2022-11-11,3,Average
177,178,Alice,50,F,50000,Finance,2021-03-20,4,Average
81,82,Bob,50,F,80000,Finance,2020-01-15,3,Average
91,92,Bob,45,M,70000,Finance,2022-11-11,4,Good
52,53,Alice,25,M,40000,IT,2020-01-15,1,Needs Improvement
19,20,David,20,M,50000,IT,2022-11-11,1,Good
190,191,David,25,M,40000,IT,2020-01-15,4,Average
157,158,Alice,35,M,50000,IT,2020-01-15,1,Excellent
